In [0]:
### We are going to train a model with huge data of tweets and predict the sentiment of the tweet
#### 0-Negative 2-Neutral and 4- positive
#### For word embedding and weights we will load from standford file. For every word it has weights with 100 dimensions
#### So vocan size would be all words in the training set.

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv


--2020-05-28 18:06:09--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.128, 2607:f8b0:4001:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M   112MB/s    in 2.0s    

2020-05-28 18:06:11 (112 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [0]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
import csv
import json

In [0]:
corpus=[]
num_sentences=0
with open('/tmp/training_cleaned.csv','r') as csvfile:
  csv_data=csv.reader(csvfile,delimiter=',')
  for data in csv_data:
    ### We will take only first item and 6th item from the list of row
    list_row=[]
    list_row.append(data[0])
    list_row.append(data[5])
    corpus.append(list_row)
    num_sentences +=1
    


In [0]:
print(len(corpus))
print(num_sentences)
corpus[1][1]

1600000
1600000


"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [0]:
### Now lets start training

In [0]:
sentences=[]
labels=[]
for label,cat in corpus:
  sentences.append(cat)
  labels.append(int(label))

In [0]:
len(labels)

1600000

In [0]:
### WE will split the data 70-30
from sklearn.model_selection import train_test_split

In [0]:
train_data,test_data,train_labels,test_labels=train_test_split(sentences,labels,test_size=0.2)

In [0]:
len(train_data),len(test_data)

(1280000, 320000)

In [0]:
### Tokenization

In [0]:
embed_dim=100
max_len=16
trunc_text='post'
padding='post'
oov_token='<OOV>'

In [0]:
 from tensorflow.keras.preprocessing.text import Tokenizer

In [0]:
#### we won't assign weights here vocab size here because we have another 
token=Tokenizer(oov_token=oov_token)
token.fit_on_texts(train_data)
word_index=token.word_index
## now vocab size would be every word in the word_index
vocab_size=len(word_index)
train_seq=token.texts_to_sequences(train_data)
test_seq=token.texts_to_sequences(test_data)

In [0]:
### Padding Sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
train_pad_seq=pad_sequences(train_seq,maxlen=max_len,padding=padding,truncating=trunc_text)
test_pad_seq=pad_sequences(test_seq,maxlen=max_len,padding=padding,truncating=trunc_text)

In [0]:
### Now we have done pre processing.
### For embedded weights we will take weights from other sources instead of creating on our own with embedding layer
### We are taking from standford Glove dataset. Where it has words weights of 100 dimensions. that is the reason we also defined the wmbed_dim
### of 100 dimensions.

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt

--2020-05-28 18:07:22--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M  55.4MB/s    in 13s     

2020-05-28 18:07:36 (24.5 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [0]:
### first we will map weights for all the words
### Then we will create an empty array with zeros and shape of the weights and then we will assign weights present in our word_index

In [0]:

with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8',) as filedef:
  for lines in filedef:
    print(type(lines))
    print
    break


<class 'str'>


In [0]:
### So the line is string with space seperated and first value is word and rest are weights. 
### We will split and add the weights accordingly.


In [0]:
embedded_index={}
with open('/tmp/glove.6B.100d.txt','r',encoding='utf-8') as glove:
  for lines in glove:
    lines_weights=lines.split()
    word=lines_weights[0]
    weights=np.asarray(lines_weights[1:], dtype=np.float32)
    embedded_index[word]=weights

In [0]:
embedded_index['the']

array([-0.038194, -0.24487 ,  0.72812 , -0.39961 ,  0.083172,  0.043953,
       -0.39141 ,  0.3344  , -0.57545 ,  0.087459,  0.28787 , -0.06731 ,
        0.30906 , -0.26384 , -0.13231 , -0.20757 ,  0.33395 , -0.33848 ,
       -0.31743 , -0.48336 ,  0.1464  , -0.37304 ,  0.34577 ,  0.052041,
        0.44946 , -0.46971 ,  0.02628 , -0.54155 , -0.15518 , -0.14107 ,
       -0.039722,  0.28277 ,  0.14393 ,  0.23464 , -0.31021 ,  0.086173,
        0.20397 ,  0.52624 ,  0.17164 , -0.082378, -0.71787 , -0.41531 ,
        0.20335 , -0.12763 ,  0.41367 ,  0.55187 ,  0.57908 , -0.33477 ,
       -0.36559 , -0.54857 , -0.062892,  0.26584 ,  0.30205 ,  0.99775 ,
       -0.80481 , -3.0243  ,  0.01254 , -0.36942 ,  2.2167  ,  0.72201 ,
       -0.24978 ,  0.92136 ,  0.034514,  0.46745 ,  1.1079  , -0.19358 ,
       -0.074575,  0.23353 , -0.052062, -0.22044 ,  0.057162, -0.15806 ,
       -0.30798 , -0.41625 ,  0.37972 ,  0.15006 , -0.53212 , -0.2055  ,
       -1.2526  ,  0.071624,  0.70565 ,  0.49744 , 

In [0]:
### Now we have got the weights for each word. 
### We will loop the same with our word_index and then we will assign those weights to out words
### we will create an array with zeros and then we will asssign weights which maps with rhe standford glove words

In [0]:
embed_vector=np.zeros((vocab_size+1,100),dtype=np.float32)

In [0]:
embed_vector.shape

(595155, 100)

In [0]:
for words,i in word_index.items():
  # We will search the word in the embed index and if we find then we will takthe index and assign the weights
  word_vector=embedded_index.get(words)
  
  if word_vector is not None:
    embed_vector[i] = embedded_index[words]
    #break
  


In [0]:
len(word_index)

595154

In [0]:
embed_vector.shape,embed_vector[594436]

((595155, 100),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32))

In [0]:
vocab_size

595154

In [0]:
### Now we have got all the weights ffor word index which we usually used to get fropm Embedding layer.
### Now we will create model in that , in Embedding layer we will pass these weights of the word index and turn off
### the train mode for that particular layer

In [0]:
from tensorflow import keras
from tensorflow.keras.layers import Embedding,Dense,GlobalAveragePooling1D,Flatten

In [0]:
model=keras.Sequential()
model.add(Embedding(input_dim=vocab_size+1,output_dim=embed_dim,input_length=max_len,weights=[embed_vector],trainable=False))
model.add(Flatten())
model.add(Dense(units=6,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 16, 100)           59515500  
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 9606      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 59,525,113
Trainable params: 9,613
Non-trainable params: 59,515,500
_________________________________________________________________


In [0]:
train_labels=np.asarray(train_labels)
test_labels=np.array(test_labels)

In [0]:
for num in train_labels:
  if num==2:
    print(num)
    break

In [0]:
model.fit(train_pad_seq,train_labels,epochs=10,validation_data=(test_pad_seq,test_labels))

Epoch 1/10
40000/40000 [==============================] - 65s 2ms/step - loss: -45411.9336 - accuracy: 4.2969e-05 - val_loss: -129603.4844 - val_accuracy: 0.0000e+00
Epoch 2/10
40000/40000 [==============================] - 62s 2ms/step - loss: -289011.4688 - accuracy: 0.0000e+00 - val_loss: -494419.1875 - val_accuracy: 0.0000e+00
Epoch 3/10
40000/40000 [==============================] - 63s 2ms/step - loss: -782520.5000 - accuracy: 0.0000e+00 - val_loss: -1125557.5000 - val_accuracy: 0.0000e+00
Epoch 4/10
40000/40000 [==============================] - 64s 2ms/step - loss: -1557981.0000 - accuracy: 0.0000e+00 - val_loss: -2058498.2500 - val_accuracy: 0.0000e+00
Epoch 5/10
40000/40000 [==============================] - 62s 2ms/step - loss: -2639106.7500 - accuracy: 0.0000e+00 - val_loss: -3296571.2500 - val_accuracy: 0.0000e+00
Epoch 6/10
40000/40000 [==============================] - 64s 2ms/step - loss: -4022284.7500 - accuracy: 0.0000e+00 - val_loss: -4840238.5000 - val_accuracy: 0.0